In [ ]:
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image
import random

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Ignore Warnings

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="urllib3")

print ('modules loaded')

In [ ]:
import cv2
import os

# Path to the main folder containing images
base_folder = "/Users/kisanetgabreselassie/Downloads/archive 2"

# Define the sets of folders to iterate over
sets = ['test', 'train', 'valid']
categories = ['nowildfire', 'wildfire']

# Loop over each set and category to read images
for dataset in sets:
    for category in categories:
        category_path = os.path.join(base_folder, dataset, category)

        # Load all images from the category folder
        if os.path.exists(category_path):
            image_files = os.listdir(category_path)
            for image_file in image_files:
                image_path = os.path.join(category_path, image_file)

                # Read and display image
                image = cv2.imread(image_path)

                # Check if the image was loaded correctly
                if image is not None:
                    print(f"Image: {image_file} from {dataset}/{category}")
                    print(f"Dimensions: {image.shape[:2]} (Height x Width)")
                    print(f"Channels: {image.shape[2] if len(image.shape) == 3 else 1}")
                    # Uncomment the next lines to display each image
                    # cv2.imshow("Image", image)
                    # cv2.waitKey(0)
                else:
                    print(f"Could not load image {image_file} from {dataset}/{category}")

# Close any open image windows
cv2.destroyAllWindows()


Image: -73.89161,45.573836.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -73.473177,45.459197.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -113.922643,51.09118.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -73.743496,45.478276.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -79.660026,43.850506.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -114.1557,50.913325.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -114.112213,51.071833.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -73.642351,45.563128.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -73.579539,45.544886.jpg from test/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -114.146766,51.146134.jpg from test/nowildfire
Di

Premature end of JPEG file


Image: -75.8308,48.2628.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -75.33665,49.32015.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -71.47579,48.03327.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -72.47538,48.88909.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -69.9464,49.1883.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -71.75463,51.06025.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -73.45839,54.30396.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -79.43172,48.80756.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -77.2131,54.20174.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -68.47195,47.68645.jpg from test/wildfire
Dimensions: (350, 350) (Height x Width)
Channel

Premature end of JPEG file


Image: -113.959818,51.078172.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -75.694063,45.441218.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -75.654104,45.405802.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -122.958384,49.180341.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -79.400652,43.851258.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -123.110478,49.139372.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -73.803234,45.61231.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -114.168733,50.990151.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -73.849881,45.569532.jpg from train/nowildfire
Dimensions: (350, 350) (Height x Width)
Channels: 3
Image: -79.503458,43.721246.jpg from train/

In [4]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the base folder path
base_folder = "/Users/kisanetgabreselassie/Downloads/archive 2"

# Path to datasets
train_dir = os.path.join(base_folder, 'train')
valid_dir = os.path.join(base_folder, 'valid')
test_dir = os.path.join(base_folder, 'test')

# Set up ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from train and validation directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust based on your model input size
    batch_size=32,
    class_mode='binary'  # Assuming binary classification (wildfire vs nowildfire)
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

print("Data Generators ready for training.")


Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Data Generators ready for training.
